### Movie Recommendation System

In [59]:
import numpy as np
import pandas as pd

In [60]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [61]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [62]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [63]:
# merging movies and cast
movies = movies.merge(credits,on='title')

#### 
tags for system: 
genres, id, keywords, title, overview, cast, crew

In [64]:
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [65]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [66]:
movies.columns

Index(['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew'], dtype='object')

### >> pre-processing

In [67]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [68]:
movies.dropna(inplace=True)

In [69]:
movies.duplicated().sum()

np.int64(0)

In [70]:
movies.iloc[0].genres
# it is giving string, so we have to convert str to into a real Python list of dictionaries.

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [71]:
import ast
def convert(obj):
    L = []
    for val in ast.literal_eval(obj):
        L.append(val['name'])
    return L

In [72]:
movies['genres'] = movies['genres'].apply(convert)

In [73]:
movies['keywords'] = movies['keywords'].apply(convert)

In [74]:
def convertTopThreeCast(obj):
    L = []
    counter=0
    for stringVal in ast.literal_eval(obj):
        if counter < 3:
            L.append(stringVal['name'])
            counter+=1
        else:
            break
    return L 


In [75]:
movies['cast'] = movies['cast'].apply(convertTopThreeCast) 

In [76]:
def crewDirectorName(obj):
    L = []
    for val in ast.literal_eval(obj):
        if val['job']=='Director':
            L.append(val['name'])
            break
    return L

In [77]:
movies['crew'] = movies['crew'].apply(crewDirectorName)

In [78]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [79]:
movies['genres'] = movies['genres'].apply(lambda x :[i.replace(" ", "") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x :[i.replace(" ", "") for i in x])
movies['cast'] = movies['cast'].apply(lambda x :[i.replace(" ", "") for i in x])
movies['crew'] = movies['crew'].apply(lambda x :[i.replace(" ", "") for i in x])

In [81]:
movies['tags'] = movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [84]:
new_df = movies[['movie_id','title','tags']]

In [86]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\princ\AppData\Local\Temp\ipykernel_14240\3089450492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))


In [92]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

C:\Users\princ\AppData\Local\Temp\ipykernel_14240\3214958533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())
